In [ ]:

import numpy as np
from cvxpy import *

# Linear Program

In [ ]:
# p = Parameter(sign="positive", value=4)
p = 4
u = np.array([0.3, 0.25, 0.45])
t = np.array([5.0,6.0,2.0]) #.reshape((i,1))
# k = Parameter(sign="positive", value=3)
k = 3
# T = Parameter(sign="positive", value=10)
T = 10

X = Variable(k,p)
r = Variable(k)
q = Variable(k)

probs = []
for i in range(k):
#     obj = q[i]*u[i] - r[i]
    constr = [q[i] == sum_entries(X[i,:])*t[i]/T - r[i],
              q[i] >= 0,
              q[i] <= 1,
              r[i] >= 0]
    probs.append(Problem(Maximize(q[i]*u[i] - r[i]), constr))

full_prob = sum(probs)
for j in range(p): 
    # should this be equality or lt_equals?
    full_prob.constraints += [sum_entries(X[:,j]) == 1]
full_prob.constraints += [sum_entries(X) == p]
# print [c.is_dcp() for c in full_prob.constraints]

full_prob.solve()
X.value



# total_thru = mul_elemwise(t/T, sum_entries(X, axis=1))
# constraints = []




# constraints.append(sum_entries(X,axis=0) == 1) # slot used constraint
# constraints.append(sum_entries(X) == p) # total_slot_constraint
# constraints.append(q >= 0)
# constraints.append(q <= 1)
# constraints.append(r >= 0)
# # constraints.append(q == total_thru - r)
# obj = Maximize(sum_entries(mul_elemwise(q,u)) - sum_entries(r))
# prob = Problem(obj, constraints)
# prob.solve()






# print total_thru.size

In [ ]:
print r.value
print q.value

# Integer Program

In [ ]:
p = 4
u = np.array([0.3, 0.25, 0.45])
t = np.array([5.0,6.0,2.0]) #.reshape((i,1))
# k = Parameter(sign="positive", value=3)
k = 3
# T = Parameter(sign="positive", value=10)
T = 10

X = Int(k,p)
r = Variable(k)
q = Int(k)

maxs = []
constraints = []
for i in range(k):
#     obj = q[i]*u[i] - r[i]
#     constr = [q[i] == sum_entries(X[i,:])*t[i]/T - r[i]]
    maxs.append(Maximize(q[i]*u[i] - r[i]))
    constraints += [q[i] == sum_entries(X[i,:]) - r[i]]
    constraints += [sum_entries(X[i,:])*t[i] >= T*q[i]]



for j in range(p): 
    # should this be equality or lt_equals?
    constraints += [sum_entries(X[:,j]) == 1]
    

    
constraints += [sum_entries(X) == p]
constraints += [X <= 1]
constraints += [X >= 0]
constraints += [q <= 1]
constraints += [q >= 0]
constraints += [r >= 0]
prob = Problem(sum(maxs), constraints)


# print [c.is_dcp() for c in full_prob.constraints]

prob.solve()
xs = np.zeros((k,p))
for i in range(k):
    for j in range(p):
        cur_x = X.value[i,j]
        if cur_x > 0.0000001:
            xs[i,j] = cur_x
print xs
        
# res = [x if x > 0.0000001 else 0 for x in X.value]

In [3]:
import numpy as np
from cvxpy import *
def assign_slots(num_slots, utilities, throughputs, required_throughput):
# u = np.array([0.3, 0.25, 0.45])
# t = np.array([5.0,6.0,2.0]) #.reshape((i,1))
# # k = Parameter(sign="positive", value=3)
# k = 3
# # T = Parameter(sign="positive", value=10)
# T = 10)
    p = num_slots
    k = len(utilities)
    assert len(throughputs) == len(utilities)
    T = required_throughput
    X = Int(k,p)
    r = Variable(k)
    q = Int(k)
    maxs = []
    constraints = []
    for i in range(k):
        maxs.append(Maximize(q[i]*u[i] - r[i]))
        constraints += [q[i] == sum_entries(X[i,:]) - r[i]]
        constraints += [sum_entries(X[i,:])*t[i] >= T*q[i]]
    for j in range(p): 
        constraints += [sum_entries(X[:,j]) == 1]
    constraints += [sum_entries(X) == p]
    constraints += [X <= 1]
    constraints += [X >= 0]
    constraints += [q <= 1]
    constraints += [q >= 0]
    constraints += [r >= 0]
    prob = Problem(sum(maxs), constraints)
    prob.solve()
    # filter small floats to 0
    xs = np.zeros((k,p))
    for i in range(k):
        for j in range(p):
            cur_x = X.value[i,j]
            if cur_x > 0.0000001:
                xs[i,j] = cur_x
    return xs

In [5]:
p = 4
u = np.array([0.3, 0.25, 0.45])
t = np.array([5.0,6.0,2.0]) #.reshape((i,1))
# k = Parameter(sign="positive", value=3)
k = 3
# T = Parameter(sign="positive", value=10)
T = 10
print (assign_slots(p,u,t,T))

[[ 0.  0.  1.  1.]
 [ 1.  1.  0.  0.]
 [ 0.  0.  0.  0.]]
